Sort and remove exact matches


In [ ]:
#This part was copied from Google Colab, but I think everything should still work fine
import pandas as pd

truth_csv_path = '/content/drive/MyDrive/summerwowow/truthv1.csv' #<--- set this to hand labled csv
guess_csv_path = '/content/drive/MyDrive/summerwowow/round2_infer_moonshot_llama_3.1_8B.csv'  #<--- set this to the llm results csv
output_path = '/content/drive/MyDrive/summerwowow/fullmismatches_round2.csv'

COLUMNS_TO_COMPARE = [2, 3, 4, 5]

df_truth = pd.read_csv(truth_csv_path)
df_guess = pd.read_csv(guess_csv_path)

# Convert all string values in truth and guess dataframes to lowercase for case-insensitive comparison
print("Lowercasing string values in truth and guess dataframes...")
df_truth = df_truth.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df_guess = df_guess.applymap(lambda x: x.lower() if isinstance(x, str) else x)

# Assert that all three dataframes have the same shape for consistent indexing
assert df_truth.shape == df_guess.shape, "Truth and Guess CSV files must have the same shape for comparison."

mismatches = []
for row_idx in range(1, df_truth.shape[0]):
    for col_idx in COLUMNS_TO_COMPARE:
        val_truth = df_truth.iat[row_idx, col_idx]
        val_guess = df_guess.iat[row_idx, col_idx]

        if val_truth != val_guess:
            mismatch_value_from_guess_values = df_guess_values.iat[row_idx, col_idx]

            mismatches.append({
                'row': row_idx + 1, 
                'column_idx': col_idx, 
                'truth_value': val_truth,
                'llm_value': val_guess,
                'mismatch_value': mismatch_value_from_guess_values # New column with value from guess_values
            })

mismatch_df = pd.DataFrame(mismatches)
print(mismatch_df.head(10000))
mismatch_df.to_csv(output_path, index=False)
print("saved to {output_path}")

Prompts and setup stuff


In [ ]:
base_prompt_template = """You are a domain-aware evaluator that classifies the relationship between a predicted answer and the actual (reference) answer from genomics studies data.


Use these standardized mismatch labels:

2. Terminology Difference – The predicted and actual answers refer to the exact same item using a different term
3. Generalization – The predicted answer is more general than the actual answer.
4. Too Specific – The predicted answer is more specific than the actual answer.
5. Subset of Truth – The predicted answer is only part of the actual answer; doesnt include all of the actual answers. there must be multiple possible answers denoted by a "/" or a ";" or ","
6. Incorrect / Out of Scope / Hallucinated – The predicted answer is incorrect, unrelated, or introduces a false concept.

Below are some examples for this task:

{few_shot_examples}

You are looking at a case with {type_name} data.

Step 1: Compare the prediction and the actual answer. What are the differences?
Step 2: Based on those differences, classify the mismatch as type 2–6.

Predicted: {predicted}
Actual: {actual}


Make sure nothing else is returned, only the number of the mismatch categor, which can be numbers 2, 3, 4, 5, or 6 (not 1). DOUBLE CHECK ITS ONLY THE NUMBER OF THE MISMATCH, only numbers should be returned, no words and no explainations. the answer should be formatted like this: "2" or "3" or "4" or "5" or "6" ONLY with no following text"""


few_shot_bank = {
    '2': [
        ("Predicted: ovarian tumor\nActual: ovarian cancer", "2: Terminology Difference – 'tumor' and 'cancer' are used interchangeably here, with no change in meaning."),
        ("Predicted: scrna-seq\nActual: snrna-seq", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: rna-seq\nActual: scrna-seq", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: scrna-seq\nActual: scrna-seq; vdj-seq", "5: Subset of Truth – prediction does not include all details from truth, or is a component of truth."),
        ("Predicted: sctcr-seq\nActual: scrna-seq; vdj-seq", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: merip-seq\nActual: plumage", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: scrna-seq\nActual: cite-seq", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
    ],
    '4': [
        ("Predicted: ovarian carcinoma\nActual: ovarian cancer", "2: Terminology Difference – different terms for the same disease."),
        ("Predicted: subq. tumor\nActual: subcutaneous tumor", "2: Terminology Difference – abbreviation does not affect the meaning."),
        ("Predicted: cervical lymph node\nActual: lymph node", "4: Too specific – prediction adds body region."),
        ("Predicted: colorectal carcinoma\nActual: colon", "4: Too specific – prediction adds a cancer label."),
        ("Predicted: pancreatic neuroendocrine tumor\nActual: pancreas", "4: Too specific – prediction adds a type of cancer."),
        ("Predicted: gastric wall\nActual: stomach", "4: Too specific – prediction adds a body part detail."),
        ("Predicted: hepatocellular carcinoma\nActual: liver", "4: Too specific – prediction adds a disease name."),
        ("Predicted: lung\nActual: lung adenocarcinoma", "3: Generalization – predicted term is broader."),
        ("Predicted: liver\nActual: hepatocellular carcinoma biopsy", "3: Generalization – predicted term is broader."),
        ("Predicted: bone marrow\nActual: spleen", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: prostate cancer\nActual: prostate", "4: Too specific – prediction has an extra detail not noted in the actual answer"),
        ("Predicted: prostate\nActual: prostate carcinoma", "3: Generalization – predicted term is broader."),
        ("Predicted: breast tumor\nActual: breast", "4: Too specific – prediction has an extra detail not noted in the actual answer"),
        ("Predicted: embryonic kidney\nActual: kidney", "4: Too specific – prediction has an extra detail not noted in the actual answer"),
        ("Predicted: bone marrow\nActual: blood", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
    ],
    '5': [
        ("Predicted: lung squamous cell carcinoma cells\nActual: lung squamous cell carcinoma", "2: Terminology Difference – adding 'cells' doesn't change the entity."),
        ("Predicted: aspc-1\nActual: aspc1", "2: Terminology Difference – same cell line with a different naming style."),
        ("Predicted: rat lung adenocarcinoma cell line\nActual: rat lung adenocarcinoma cells", "2: Terminology Difference – 'cell line' and 'cells' are interchangeable in this context."),
        ("Predicted: immortalized human dermal fibroblast line\nActual: immortalized human dermal fibroblasts", "2: Terminology Difference – 'line' is included but doesn’t change the meaning."),
        ("Predicted: primary mouse mammary fibroblasts line\nActual: primary mouse mammary fibroblasts", "2: Terminology Difference – 'line' is added, but it doesn’t meaningfully change the entity."),
        ("Predicted: kp lung tumor cells\nActual: kp cells", "2: Terminology Difference – the added descriptors don’t change the identity of the cell line."),
        ("Predicted: gata3\nActual: gata3-expressing cells", "3: Generalization – predicted term is broader."),
        ("Predicted: hek-ter\nActual: hek cells", "4: Too specific – prediction has an extra detail not noted in the actual answer"),
        ("Predicted: h1650\nActual: h1651", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: hoxb8\nActual: hoxb8 cell", "3: Generalization – predicted term is broader."),
        ("Predicted: no\nActual: hoxb8 cell", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: d458\nActual: d459", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: myc-cap\nActual: myc-cap cells", "3: Generalization – predicted term is broader."),
    ],
    '3': [
        ("Predicted: mus musculus\nActual: mus musculus; synthetic construct", "5: Subset of Truth – prediction does not include all details from truth, or is a component of truth."),
        ("Predicted: homo sapiens\nActual: synthetic construct; homo sapiens and murine mouse model", "3: Generalization – predicted term is broader."),
    ],
}

column_type_map = {
    2: "Sequencing type",
    3: "Organism type",
    4: "Tissue type",
    5: "Cell line type",
    6: "Disease type"
}


def build_prompt(row):
    task = str(row[1])                 #Column 2 is case type
    predicted = str(row[2])              #Column 3 is predicted
    actual = str(row[3])                     #Column 4 is actual
    column_idx = int(row['column_idx'])

    type_name = column_type_map.get(column_idx, "feature")

    # Format examples
    examples = few_shot_bank[task]
    few_shot_str = "\n\n".join([
        f"{ex[0]}\n{ex[1]}" for ex in examples
    ])

    return base_prompt_template.format(
        few_shot_examples=few_shot_str,
        predicted=predicted,
        actual=actual,
        type_name=type_name,
    )

Self-consistency + few shot + re-evaluation


In [ ]:
import pandas as pd
import requests
import random
import re
from collections import Counter

# === Config ===
INPUT_CSV = "/Users/yjyou/Documents/summer25/updated_mismatch_with_errors3.csv"  #<---- csv generated from the first part
OUTPUT_CSV = "/Users/yjyou/Documents/summer25/newest_round3.csv"
OLLAMA_API_URL = "http://localhost:11434/api/generate"
MODEL = "gemma3:27b"
SC_SAMPLES = 5
SC_TEMPERATURE = 0.95
ON_FAIL_FALLBACK = "6"

# === Label Extractor ===
digit_re = re.compile(r"\b([2-6])\b")
def extract_label(text):
    if not isinstance(text, str): return ON_FAIL_FALLBACK
    match = digit_re.search(text.strip())
    return match.group(1) if match else ON_FAIL_FALLBACK

mismatch_df = pd.read_csv(INPUT_CSV)
results = []
start_idx = 0
end_idx = len(mismatch_df)  # or 176 if you're limiting
print(f"🔍 Processing rows {start_idx} to {end_idx - 1}")

for i, row in mismatch_df.iloc[start_idx:end_idx].iterrows():
    prompt = build_prompt(row)  # Assumes you defined this already
    if prompt is None:
        results.append("Skipped – no examples")
        continue

    votes = []
    for _ in range(SC_SAMPLES):
        try:
            seed = random.randint(1, 1_000_000)
            response = requests.post(
                OLLAMA_API_URL,
                json={
                    "model": MODEL,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": SC_TEMPERATURE,
                        "seed": seed,
                        "top_k": 80,
                        "top_p": 1,
                    }
                },
                timeout=60
            )
            response.raise_for_status()
            content = response.json()["response"].strip()
            label = extract_label(content)
        except Exception as e:
            print(f"⚠️ Error on row {i}: {e}")
            label = ON_FAIL_FALLBACK
        votes.append(label)

    final_label = Counter(votes).most_common(1)[0][0]
    results.append(final_label)

# === Save Initial Predictions ===
mismatch_df = mismatch_df.iloc[start_idx:end_idx].copy()
mismatch_df['llm_category'] = results
mismatch_df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Self-consistency + few-shot results saved to: {OUTPUT_CSV}")

# === Reevaluate 3 vs 4 (plus 2) ===
column_type_map = {
    2: "Sequencing type",
    3: "Organism type",
    4: "Tissue type",
    5: "Cell line type",
    6: "Disease type"
}

def build_re_evaluation_prompt(row):
    """Focused re-prompt to distinguish 2 vs 3 vs 4"""
    column_type = column_type_map.get(row["column_idx"], "unknown data type"),
    return f"""
You are a domain-aware evaluator that classifies the relationship between a predicted answer and the actual (reference) answer from genomics studies data.

- 2: Terminology Difference – The predicted and actual terms refer to the same concept but use different terminology. These differences may include synonyms, abbreviations, or alternative phrasing that do not alter the underlying meaning. Use this only when the prediction and ground truth are equivalent in specificity and semantics.

- 3: Generalization – The predicted term is broader or less specific than the actual term. Some important contextual or descriptive detail from the ground truth is missing in the prediction. Use this when the prediction could apply to a wider range of concepts than intended.

- 4: Too Specific – The predicted term is more detailed or narrowly defined than the actual term. The prediction includes extra qualifiers, subtypes, or technical specifications that are not present in the actual. Use this when the prediction "over-defines" the intended label.

Respond with just the number (2, 3, or 4).

Example 1:
Actual: lung adenocarcinoma
Predicted: lung
→ Answer: 3
The predicted term is more general and omits the specific diagnosis.

Example 2:
Actual: lung
Predicted: lung adenocarcinoma
→ Answer: 4
The prediction adds unnecessary detail that wasn't present in the actual.

Example 3:
Actual: ovarian carcinoma
Predicted: ovarian cancer
→ Answer: 2
These terms are medically equivalent; one is slightly more technical.

You are looking at a case with {column_type} data.

Step 1: Compare the prediction and the actual answer. What are the differences?
Step 2: Based on those differences, now evaluate:
Actual: {row['truth_value']}
Predicted: {row['llm_value']}
→ Answer:"""

print("\n🔁 Reevaluating rows with initial label 3 or 4...\n")

refine_rows = mismatch_df[mismatch_df["llm_category"].astype(str).isin(["3", "4"])]
refined_labels = []

for i, row in refine_rows.iterrows():
    prompt = build_re_evaluation_prompt(row)
    votes = []

    for _ in range(SC_SAMPLES):
        try:
            seed = random.randint(1, 1_000_000)
            response = requests.post(
                OLLAMA_API_URL,
                json={
                    "model": MODEL,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": SC_TEMPERATURE,
                        "seed": seed,
                        "top_k": 80,
                        "top_p": 1,
                    }
                },
                timeout=60
            )
            response.raise_for_status()
            content = response.json()["response"].strip()
            label = extract_label(content)
            if label in {"2", "3", "4"}:
                votes.append(label)
        except Exception as e:
            print(f"⚠️ Reevaluation error on row {i}: {e}")
            votes.append(row["llm_category"])  # fallback

    final_label = Counter(votes).most_common(1)[0][0] if votes else row["llm_category"]
    refined_labels.append(final_label)

mismatch_df.loc[refine_rows.index, "llm_category"] = refined_labels
mismatch_df.to_csv(OUTPUT_CSV, index=False)

Scoring


In [ ]:
import pandas as pd
file_path = "/Users/hannahyou/Documents/summer25roswell/important results/newest_round1.csv" 
df = pd.read_csv(file_path)

scoring = {
    1: 1.0,
    2: 1.0,
    3: 0.7,
    4: 0.7,
    5: 0.5,
    6: 0.0
}

df['score'] = df['llm_category'].map(scoring)
max_total = 736
actual_predictions = len(df) - 1  #header row
missing_rows = max_total - actual_predictions
present_score = df['score'].sum()
missing_score = missing_rows * 1.0
final_total_score = present_score + missing_score
final_average_score = final_total_score / max_total
print(f"Final Total Score (out of 736): {final_total_score}")
print(f"Final Average Score: {round(final_average_score, 3)}")

Final Total Score (out of 736): 307.4
Final Average Score: 0.418
